In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from read_data import ReadClass

rc('text', usetex=True)
rc('font', family='serif')

experiment_name = "exRo10N3e06_LES"
figure_directory = "../figure/" + experiment_name
if not os.path.isdir(figure_directory):
    os.makedirs(figure_directory)

reader = ReadClass(experiment_name)
reader.read_geometry(0)

output_per_day = 86400 / reader.output_interval

def get_mean_spectrum_HV(nt, interval, variable):
    data, K_axis, M_axis = reader.get_spectrum_HV(nt, variable)
    if interval == 0:
        return data, K_axis, M_axis
    for it in range(nt+1, nt+interval):
        tmp, K_axis, M_axis = reader.get_spectrum_HV(it, variable)
        data = data + tmp
    data = data / interval
    return data, K_axis, M_axis


def get_mean_spectrum_V(nt, interval, variable):
    data, M_axis = reader.get_spectrum_V(nt, variable)
    if interval == 0:
        return data, M_axis
    for it in range(nt+1, nt+interval):
        tmp, M_axis = reader.get_spectrum_V(it, variable)
        data = data + tmp
    data = data / interval
    return data, M_axis


def get_mean_spectrum_H(nt, interval, variable):
    data, K_axis = reader.get_spectrum_H(nt, variable)
    if interval == 0:
        return data, K_axis
    for it in range(nt+1, nt+interval):
        tmp, K_axis = reader.get_spectrum_H(it, variable)
        data = data + tmp
    data = data / interval
    return data, K_axis
    

def get_mean_spectrum_X(nt, interval, variable):
    data, KX_axis = reader.get_spectrum_X(nt, variable)
    if interval == 0:
        return data, KX_axis
    for it in range(nt+1, nt+interval):
        tmp, KX_axis = reader.get_spectrum_X(it, variable)
        data = data + tmp
    data = data / interval
    return data, KX_axis
    

def get_mean_spectrum_Y(nt, interval, variable):
    data, KY_axis = reader.get_spectrum_Y(nt, variable)
    if interval == 0:
        return data, KY_axis
    for it in range(nt+1, nt+interval):
        tmp, KY_axis = reader.get_spectrum_Y(it, variable)
        data = data + tmp
    data = data / interval
    return data, KY_axis
    

def get_mean_spectrum_1d(nt, interval, variable):
    data, K_axis = reader.get_spectrum_1d(nt, variable)
    if interval == 0:
        return data, K_axis
    for it in range(nt+1, nt+interval):
        tmp, K_axis = reader.get_spectrum_1d(it, variable)
        data = data + tmp
    data = data / interval
    return data, K_axis

In [ ]:
fs = 16

start_day = 11
interval_day = 2

nt = int(start_day * output_per_day)
interval = int(interval_day * output_per_day)

PE_V, M_axis = get_mean_spectrum_V(nt, interval, 'PE')
KE_V, M_axis = get_mean_spectrum_V(nt, interval, 'KE')

PE_X, KX_axis = get_mean_spectrum_X(nt, interval, 'PE')
KE_X, KX_axis = get_mean_spectrum_X(nt, interval, 'KE')

PE_Y, KY_axis = get_mean_spectrum_Y(nt, interval, 'PE')
KE_Y, KY_axis = get_mean_spectrum_Y(nt, interval, 'KE')

NKE, K1d_axis = get_mean_spectrum_1d(nt, interval, 'NKE')
NKE = NKE / reader.output_interval
NPE, K1d_axis = get_mean_spectrum_1d(nt, interval, 'NPE')
NPE = NPE / reader.output_interval
DKE, K1d_axis = get_mean_spectrum_1d(nt, interval, 'DKE')
DKE = DKE / reader.output_interval
DPE, K1d_axis = get_mean_spectrum_1d(nt, interval, 'DPE')
DPE = DPE / reader.output_interval
CKP, K1d_axis = get_mean_spectrum_1d(nt, interval, 'CKP')
CKP = CKP / reader.output_interval

KE, K1d_axis = get_mean_spectrum_1d(nt, interval, 'KE')
PE, K1d_axis = get_mean_spectrum_1d(nt, interval, 'PE')

DKE_Flux = np.append(np.cumsum(DKE)[::-1], 0)[::-1]
DPE_Flux = np.append(np.cumsum(DPE)[::-1], 0)[::-1]
NKE_Flux = np.append(np.cumsum(NKE[::-1])[::-1], 0)
NPE_Flux = np.append(np.cumsum(NPE[::-1])[::-1], 0)
CKP_Cum = np.append(np.cumsum(CKP)[::-1], 0)[::-1]
K1d_extend = np.insert(K1d_axis, 0, - K1d_axis[1]) + K1d_axis[1] / 2

fig = plt.figure(figsize=[6, 5])
fig.subplots_adjust(left=0.18, bottom=0.15, right=0.96,
                    top=0.9, wspace=0.2, hspace=0.2)

ax1 = fig.add_subplot(111)
ax1.plot(KX_axis, KE_X, 'r-', label=r'$\hat{E}_u$ v.s. $k$')
ax1.plot(KY_axis, KE_Y, 'r--', label=r'$\hat{E}_u$ v.s. $\ell$')
ax1.plot(M_axis, KE_V, 'r:', label=r'$\hat{E}_u$ v.s. $m$')
ax1.plot(KX_axis, PE_X, 'b-', label=r'$\hat{E}_\theta$ v.s. $k$')
ax1.plot(KY_axis, PE_Y, 'b--', label=r'$\hat{E}_\theta$ v.s. $\ell$')
ax1.plot(M_axis, PE_V, 'b:', label=r'$\hat{E}_\theta$ v.s. $m$')

M_short_axis = np.linspace(0.15, 0.6)
ax1.plot(M_short_axis, 3.e-6 * M_short_axis**(-5/3), 'k')
ax1.legend(fontsize=14)

ax1.set_ylim(1.e-9, 1.e-2)

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.grid(which='minor')

ax1.set_xlabel(r'Wavenumber $k, \ell, m$ [rad/m]', fontsize=fs)
ax1.set_ylabel(r'Energy density [J$\cdot$m/kg]', fontsize=fs)

figure_name = figure_directory + '/' +  'energy_spectrum'
fig.savefig(figure_name + '.eps')
fig.savefig(figure_name + '.png')